In [61]:
W = WeylGroup(["G",2,1], prefix="s")
[s0,s1,s2] = W.simple_reflections()


e = W(1)

MinCell = [
    s0, s0*s2*s1*s2*s0, s0*s2*s1*s2*s1*s2*s0,
    s0*s2, s0*s2*s1*s2, s0*s2*s1*s2*s1*s2,
    s0*s2*s1, s0*s2*s1*s2*s1,
    ##############
    s2*s0, s2*s1*s2*s0, s2*s1*s2*s1*s2*s0,
    s2, s2*s1*s2, s2*s1*s2*s1*s2,
    s2*s1, s2*s1*s2*s1,
    ##############
    s1*s2*s0, s1*s2*s1*s2*s0,
    s1*s2, s1*s2*s1*s2,
    s1, s1*s2*s1, s1*s2*s1*s2*s1  
]

W_small = []
for x in MinCell:
    W_small += W.bruhat_interval(e,x)
W_small = list(set(W_small))


####### Kazhdan-Lusztig polynomials ##########################

R.<q> = LaurentPolynomialRing(QQ)

KL = KazhdanLusztigPolynomial(W,q)  # KL-polynomials implemented in standard Sage way
# http://doc.sagemath.org/html/en/reference/combinat/sage/combinat/kazhdan_lusztig.html

def KLP(x,y):
    return KL.P(x,y)


def mu(w,x):    
    
    if w.bruhat_le(x):
 
        poly_dict = KLP(w,x).dict()       
        j = (x.length()-w.length()-1)/2 
        if j not in poly_dict.keys():
            return 0
        return poly_dict[j]

    return 0







######### Some operations on graded characters #########


def remove_keys_with_value(dict,value):
    '''Removes all items from dict with given value. Changes the original dict, does not return anything.
    Auxiliary function used in GradedChar().remove_zeros.'''
    
    remove = []
    for x in dict:
        if dict[x] == value:
            remove.append(x)
            
    for x in remove:
        dict.pop(x)

def remove_keys_with_value_smaller_than(dict,bound):
    '''Removes all items from dict with value smaller than or equal to "bound".
    Changes the original dict, does not return anything.
    Auxiliary function used in GradedChar().only_positive.'''
    
    remove = []
    for x in dict:
        if dict[x] <= bound:
            remove.append(x)
            
    for x in remove:
        dict.pop(x)

    
def format_star(string):
    '''Windows cannot have * in filenames, so need to use this function when saving files.
    This will replace "*" with "_".'''
    
    new_string = string.replace("*", "_")
    return new_string


def convert_to_123(w):
    '''Converts an element from W to the "123" string notation.
    Does not work with coefficients, as "convert_to_123_long".'''
    
    if w == W(1):
        return "e"
    
    return str(w).replace("s","").replace("*","")


def convert_from_123(string):
    '''Converts one element from W in the "123" string notation to the usual "s1*s2*s3" notation.'''
    
    if type(string)== Integer:
        string = str(string)
        
    if string == "e":
        return W(1)
    
    string = "*".join([char for char in string])
    
    for i in range(1,n+1):
        string = string.replace(str(i),"s%s"%i)
    
    return eval(string)


def convert_to_123_long(string):
    '''E.g. For input "4*s1*s2*s3", the output is "4*123".
    Not used for now.'''
    
    string_e = ""  # This should be a copy of string, but for "e" instead of each "1" that represent the trivial composition factor.
    
    for i in range(len(string)):
        
        condition = 0
        
        if string[i] == "1" and i>0:
            
            if string[i-1] not in ["s","-","0","1","2","3","4","5","6","7","8","9"]:
                
                if i+1==len(string):
                    condition = 1
                
                else:
                    if string[i+1] not in ["*",":","0","1","2","3","4","5","6","7","8","9"]:
                        condition = 1
    
        if condition == 0:
            string_e += string[i]
        
        else:
            string_e += "e"
    
        
    string_no_ast = re.sub(r'(?<=(?<=s).)\*',"",string_e) # Remove "*" if there is "s" two places before.
    string_no_ast_s = string_no_ast.replace("s","")
    
    return string_no_ast_s


class GradedChar:
    '''A class representing a graded character of module in the graded version of category O.
    self.graded is a dictionary where keys are indices of the graded components.
    Each graded component is a dictionary with keys being composition factors in that graded piece, and values are multiplicities.'''
    
    def __init__(self):
        self.component = {}
        self.name = ""
           
    def __str__(self):
        '''Each line becomes graded piece.'''
        
        if self == char_0():   # For zero character, print "0" instead of nothing.
            return "0"
        
        str_glob = ""
        if self.name != "":
            str_glob += self.name +":\n\n"
        
        for i in sorted(self.component.keys()):
            lis_i = []
            
            for w in (self.component[i]).keys():
                
                m = self.component[i][w]    # Multiplicity of w in i-th graded piece of self.
                m_string = ""
                if m != 1:
                    m_string = "%d*"%m      # Needed to print nicelly formated graded pieces.
                
                lis_i += ['%s'%m_string + str(w)]
            
            str_glob += "%d: "%i + ", ".join(lis_i) + "\n"
        
        return str_glob

    def __eq__(self, other): 
        if not isinstance(other, GradedChar):
            # Don't attempt to compare against unrelated types.
            return NotImplemented
        
        self.remove_zeros()      # Remove all the redundancies before comparing.
        other.remove_zeros()
        
        return self.component == other.component

    def __add__(self,char2):    
        char_sum = GradedChar()

        for k in (set(self.component.keys()) | set(char2.component.keys())):    # Union of keys without repeition.

            self_k = self.component.get(k,{})   # ".get" avoids KeyError
            char2_k = char2.component.get(k,{})

            char_sum.component[k] = {}

            for w in (set(self_k.keys()) | set(char2_k.keys())):

                char_sum.component[k][w] = self_k.get(w,0) + char2_k.get(w,0)

        return char_sum
    
    def __mul__(self,other):
        prod = GradedChar()

        for k in self.component:
            prod.component[k] = {}
        
            for w in self.component[k]:
                prod.component[k][w] = other * self.component[k][w]

        return prod
    
    def __rmul__(self,other):
        return self*other
    
    def __sub__(self,other):
        return self+((-1)*other)
    
    def __neg__(self):
        return (-1)*self
    
    
    def rename(self,new_name):
        '''Changes the name of self, and returns new_name.'''
        
        self.name = new_name
        return self.name
    
    
    def remove_zeros(self):
        '''Removes all composition factors with coefficients zero.
        These can occur only when subtracting, i.e. dealing with characters of virtual modules.'''
        
        for k in self.component:
            remove_keys_with_value(self.component[k],0)
        
        remove_keys_with_value(self.component,{})
        
        return self
        
    
    def only_positive(self):
        '''Removes all composition factors with non-positive coefficients.
        These can occur only when subtracting, i.e. dealing with characters of virtual modules.'''
        
        for k in self.component:
            remove_keys_with_value_smaller_than(self.component[k],0)
        
        remove_keys_with_value(self.component,{})
        
        return self    
    
    
    
    
    def add_factor(self,i,w):
        '''Adds a composition factor to self in i-th graded piece.'''
        
        if i not in self.component:
            self.component[i] = {}
        
        if w not in self.component[i]:
            self.component[i][w] = 0
        
        self.component[i][w] += 1
        
        self.name = ""    # The character "self" has changed, so the old name is not valid anymore.
        
        return self

    
    def is_simple(self):
        '''Checks whether self is a simple module.'''

        self.remove_zeros()

        if len(components(self)) != 1:
            return False

        only_component = self.component[ list(self.component.keys())[0] ]

        if len(only_component.keys()) != 1:
            return False

        only_factor = list(only_component.keys())[0]

        if only_component[ only_factor ]  != 1:
            return False

        return True

    
    
######### Some operations on graded characters #########


def shift(char,i):
    '''Returns the shift by i of char.'''
    
    char_new = GradedChar()
    
    for key in char.component.keys():
        char_new.component[key-i] = char.component[key]
    
    return char_new


def dual(char):
    '''Returns the graded dual of char.
    Input: GradedChar.'''
    
    char_dual = GradedChar()
    
    for key in char.component.keys():
       char_dual.component[-key] = char.component[key]
    
    return char_dual


def components(char):
    '''Returns the list of indexes of non-zero graded pieces of char.'''
    
    char.remove_zeros()
    return sorted(char.component.keys())


def truncate(char,lis):
    '''Cuts off from char all graded pieces from lis.'''
    
    for k in lis:
        char.component.pop(k, None)
    
    return char
    
    
def mult(char,i,w):
    '''Returns the multiplicity of w in i-th graded piece of char.'''
    
    if i not in char.component:
        return 0
    
    return char.component[i].get(w,0)


def total_mult(char,w):
    '''Returns the total multiplicity of w in char.'''
    
    return sum(char.component[i].get(w,0) for i in char.component)


def dict_mult(char,w):
    '''Returns dictionary with items (k:m), where m is the multiplicity of w in k-th graded piece of char.'''
    
    dic = {}
    for k in char.component:
        if char.component[k].get(w,0) != 0:
            dic[k] = char.component[k][w]
            
    return dic


def clean(char):
    '''Returns cleaner output: simple reflections are denoted by 1,2,3, ...'''
    
    char_dummy = GradedChar()
    
    char_dummy.name = (char.name).replace("(1)","(e)").replace("s","").replace("*","")
    
    for k in char.component:
        char_dummy.component[k] = {}
        
        for w in char.component[k]:
            char_dummy.component[k][convert_to_123(w)] = char.component[k][w]
    
    return(str(char_dummy))


def print_clean(char):
    '''Prints cleaner output: simple reflections are denoted by 1,2,3, ...'''
    
    print(clean(char))
    




######### Some graded characters #########


def char_0():
    '''Returns the zero graded character.'''
    
    zero = GradedChar()
    zero.rename('0')
    return zero


def char_L(w):
    '''Returns the graded character of the simple L(w).'''
    
    L = GradedChar()
    L.add_factor(0,w)
    L.rename('L(%s)'%w)
    return L

        




################ Projective functors ################


def theta_simple_simple(s,y):        
    '''Returns the graded character of theta_s(L_y), where s is a simple reflection,
    and y an element from W (not of class GradedChar).
    Reference: Coulembier-Mazorchuk-Zhang, Proposition 16,
    plus the fact that y and z are comparable if z appears.'''
        
    result = GradedChar()
    
    if y.bruhat_le(y*s):
        return result
    
    result.add_factor(-1,y)
    result.add_factor(1,y)
    result.add_factor(0,y*s)
    
    for z in W_small:
        
        if  y.bruhat_le(z) and z.bruhat_le(z*s):
            m = mu(y,z)
            if m != 0:
                result += m*char_L(z)
    
    return result


def theta_simple(s,M):
    '''Returns the graded character of theta_s(M), where s is a simple reflection,
    and M is from class GradedChar.
    Uses the function "theta_simple_simple" and the fact that theta is exact functor.'''
        
    result = GradedChar()
    
    for k in M.component:
        for w in M.component[k]:
            result += shift(M.component[k][w] * theta_simple_simple(s,w),  -k)
    
    return result


def theta(ws, char):          
    '''Returns the graded character of theta_ws(char).
    Uses recursion from [Mazorchuk: Some homolgical properties ... I, the proof of Theorem 11.a)].'''
    
    result = GradedChar()
    
    if char.name != "":
        const_name = 'theta_%s(%s)'%(ws,char.name)

    factors = ws.reduced_word()
    
    if len(factors) == 0:    # theta_e = Identity
        return char
    
    s = W.simple_reflections()[factors[len(factors)-1]]
    
    if len(factors) == 1:    # basis of the recursion
        
        result = theta_simple(s,char)
        
        if char.name != "":
            result.rename(const_name)
        return result
    
    w = ws * s
        
    subtract = [ y for y in W.bruhat_interval(e,w) if (y*s).bruhat_le(y) and mu(y,w) !=0  ]
    
    result = theta_simple(s, theta(w,char))
    
    for y in subtract:
        result = result - mu(y,w)*theta(y,char)
    
    if char.name != "":
        result.rename(result.rename(const_name))
    return result


def M(x,y):
    '''Different notation for theta_x(L(y)).'''
    
    return theta(x,char_L(y))


def th(a,b):
    '''Same as theta, but uses 123-notation in number format, not as strings.
    Also prints theta along the way.'''
    
    x = convert_from_123(str(a))
    y = convert_from_123(str(b))
    
    X = theta(x,char_L(y))
    print_clean(X)

    return  X





    


def DR(w):
    '''Returns the set of simple right descents of w.'''
    
    return {W.simple_reflections()[i] for i in w.descents()}


def DL(w):
    '''Returns the set of simple left descents of w.'''
        
    return DR(w.inverse())





In [60]:
DynkinDiagram(W)

  3
O=<=O---O
1   2   0   
G2~

In [68]:
thetas = []

for x in MinCell:
    for y in MinCell:
        X = M(x,y)
        if X != char_0():
            if X not in thetas:
                thetas.append(X)

In [69]:
thetas

In [108]:

def Equal_Products():
    '''Returns a pair of two lists L1 and L2 of the same length.
    For each i, L1[i] is the set of pairs (x,y) such that theta_x(L(y)) == L2[i].
    L2 consists of all mutually distinct non-trivial charaters that can be obtained as theta_x(L(y)),
    excluding the simple modules ( theta_e(L(y)) ) and the tilting modules ( theta_x(L(w0)) ).
    
    Works for A4 and B3, but not above.'''
    
    table = ([],[])

    for x in MinCell:
        for y in MinCell:


            X = M(x,y)
            X.name =""

            if X != char_0():

                if X in table[1]:

                    ind = (table[1]).index(X)
                    table[0][ind].append( (x,y) )

                else:

                    (table[1]).append(X)
                    (table[0]).append([(x,y)])
    return table         


In [109]:
EP = Equal_Products()

In [111]:
EP[0]

[[(s0, s0),
  (s0*s2*s1*s2*s1*s2*s0, s0*s2*s1*s2*s1*s2*s0),
  (s2*s0, s0*s2),
  (s2*s1*s2*s1*s2*s0, s0*s2*s1*s2*s1*s2)],
 [(s0, s0*s2*s1*s2*s0),
  (s0*s2*s1*s2*s0, s0),
  (s0*s2*s1*s2*s0, s0*s2*s1*s2*s1*s2*s0),
  (s0*s2*s1*s2*s1*s2*s0, s0*s2*s1*s2*s0),
  (s2*s0, s0*s2*s1*s2),
  (s2*s1*s2*s0, s0*s2),
  (s2*s1*s2*s0, s0*s2*s1*s2*s1*s2),
  (s2*s1*s2*s1*s2*s0, s0*s2*s1*s2)],
 [(s0, s0*s2*s1*s2*s1*s2*s0),
  (s0*s2*s1*s2*s1*s2*s0, s0),
  (s2*s0, s0*s2*s1*s2*s1*s2),
  (s2*s1*s2*s1*s2*s0, s0*s2)],
 [(s0, s2*s0),
  (s0*s2*s1*s2*s1*s2*s0, s2*s1*s2*s1*s2*s0),
  (s2*s0, s2),
  (s2*s1*s2*s1*s2*s0, s2*s1*s2*s1*s2)],
 [(s0, s2*s1*s2*s0),
  (s0*s2*s1*s2*s0, s2*s0),
  (s0*s2*s1*s2*s0, s2*s1*s2*s1*s2*s0),
  (s0*s2*s1*s2*s1*s2*s0, s2*s1*s2*s0),
  (s2*s0, s2*s1*s2),
  (s2*s1*s2*s0, s2),
  (s2*s1*s2*s0, s2*s1*s2*s1*s2),
  (s2*s1*s2*s1*s2*s0, s2*s1*s2)],
 [(s0, s2*s1*s2*s1*s2*s0),
  (s0*s2*s1*s2*s1*s2*s0, s2*s0),
  (s2*s0, s2*s1*s2*s1*s2),
  (s2*s1*s2*s1*s2*s0, s2)],
 [(s0, s1*s2*s0),
  (s0*s2*s1*s2*s1*s2*s

In [117]:
for i in range(len(EP[1])):
    
    equal_factors = EP[0][i]
    product = EP[1][i]
    
    print ("{ " + " , ".join(["%s * %s" %(convert_to_123(p[1]),convert_to_123(p[0]))   for p in  equal_factors ]) + " }:")
    
    print("")
    
    print_clean(product)

{ 0 * 0 , 0212120 * 0212120 , 02 * 20 , 021212 * 212120 }:

-1: 0
0: e, 02
1: 0

{ 02120 * 0 , 0 * 02120 , 0212120 * 02120 , 02120 * 0212120 , 0212 * 20 , 02 * 2120 , 021212 * 2120 , 0212 * 212120 }:

-1: 02120
0: 0212
1: 02120

{ 0212120 * 0 , 0 * 0212120 , 021212 * 20 , 02 * 212120 }:

-1: 0212120
0: 021212
1: 0212120

{ 20 * 0 , 212120 * 0212120 , 2 * 20 , 21212 * 212120 }:

-1: 20
0: 2
1: 20

{ 2120 * 0 , 20 * 02120 , 212120 * 02120 , 2120 * 0212120 , 212 * 20 , 2 * 2120 , 21212 * 2120 , 212 * 212120 }:

-1: 2120
0: 212
1: 2120

{ 212120 * 0 , 20 * 0212120 , 21212 * 20 , 2 * 212120 }:

-1: 212120
0: 21212
1: 212120

{ 120 * 0 , 12120 * 0212120 , 12 * 20 , 1212 * 212120 , 1 * 120 , 12121 * 12120 }:

-1: 120
0: 12
1: 120

{ 12120 * 0 , 120 * 0212120 , 1212 * 20 , 12 * 212120 , 12121 * 120 , 1 * 12120 }:

-1: 12120
0: 1212
1: 12120

{ 02120 * 02120 , 0212 * 2120 }:

-1: 0, 0212120, 02120
0: e, 02, 0212, 021212
1: 0, 0212120, 02120

{ 2120 * 02120 , 212 * 2120 }:

-1: 20, 2120, 212120


In [118]:
M(s1,s1*s2*s1))

theta_121(L(121)):

-1: 1, 12121, 121
0: e, 2*12, 2*1212
1: 1, 12121, 121

